In [ ]:
%load_ext autoreload
%autoreload 2

## Plan
Do differentiation of Q and V on the dfdata

In [ ]:
import os
import time
import datetime
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cellpy
from cellpy import cellreader, prms, prmreader

%matplotlib inline

In [ ]:
# Use these when working on my MacBook:
test_data_path = "/Users/jepe/scripting/cellpy/testdata"
out_data_path = "/Users/jepe/cellpy_data"

test_data_path = Path(test_data_path)
out_data_path = Path(out_data_path)

print(" SETTING SOME PRMS ".center(80, "="))
prms.Paths["db_filename"] = "cellpy_db.xlsx"
prms.Paths["cellpydatadir"] = test_data_path / "hdf5"
prms.Paths["outdatadir"] = out_data_path
prms.Paths["rawdatadir"] = test_data_path / "data"
prms.Paths["db_path"] = test_data_path / "db"
prms.Paths["filelogdir"] = test_data_path / "log"

In [ ]:
from cellpy import log
log.setup_logging(default_level="INFO")

In [ ]:
filename = Path("/Users/jepe/Arbeid/Data/celldata/20171120_nb034_11_cc.h5")
print(f"size: {filename.stat().st_size/1_048_576}")

mod_timestamp = datetime.datetime.fromtimestamp(filename.stat().st_mtime)
visited_timestamp = datetime.datetime.fromtimestamp(filename.stat().st_atime)
print(f"visited: {visited_timestamp}")
print(f"changed: {mod_timestamp}")
print()

my_data = cellreader.CellpyData()
my_data.load(filename)
dataset = my_data.dataset

In [ ]:
data = my_data.dataset
dfsummary = data.dfsummary
stept = data.step_table

In [ ]:
df = data.dfdata.copy()

In [ ]:
df.loc[df.Step_Index==6,:].head(20)

In [ ]:
r = stept.loc[(stept.cycle==1) & (stept.type=="charge"), :]

In [ ]:
r

In [ ]:
df["dq"] = 0.0
df["dv"] = np.nan

df_ch = df.Charge_Capacity.diff()
df_dch = df.Discharge_Capacity.diff()

charge_steps = stept.loc[stept.type=="charge", ["cycle", "step"]]
discharge_steps = stept.loc[stept.type=="discharge", ["cycle", "step"]]
for c, s in charge_steps.iterrows():
    cycle = s.cycle
    step = s.step
    crit = (df.Cycle_Index==cycle) & (df.Step_Index==step)
    
    df.loc[crit, "dq"] = df.loc[crit, "Charge_Capacity"].diff()
    df.loc[crit, "dv"] = df.loc[crit, "Voltage"].diff()

for c, s in discharge_steps.iterrows():
    cycle = s.cycle
    step = s.step
    crit = (df.Cycle_Index==cycle) & (df.Step_Index==step)
    
    df.loc[crit, "dq"] = df.loc[crit, "Discharge_Capacity"].diff() 
    df.loc[crit, "dv"] = df.loc[crit, "Voltage"].diff()

df["dqdv"] = df.dq / df.dv


## So far
Its possible, but to get good dqdv plots we will need to do some trix with smoothing etc.
Probably best to just iterate through all chargestep cycle-step combinations and perform a dq dv routine.

Iterating is rather slow, unfortunately...

In [ ]:
fig, ax = plt.subplots()
ax.plot(df.Voltage, df.dqdv)
ax.set_xlabel("voltage")
ax.set_ylabel("dqdv")
ax.set_xlim((0.1, 1.0))

In [ ]:
fig, ax = plt.subplots()
df.groupby("Cycle_Index").plot(x="Voltage", y="dqdv", ax=ax)